In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
from sklearn import model_selection
from sklearn.metrics import confusion_matrix

In [3]:
full_df_train = pd.read_csv("/content/drive/MyDrive/BITS/4-1/Projects/Nlp/hateval2019_en_train.csv")
full_df_test = pd.read_csv("/content/drive/MyDrive/BITS/4-1/Projects/Nlp/hateval2019_en_test.csv")
dataframe = [full_df_train,full_df_test]
full_df = pd.concat(dataframe)
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
print(full_df.head())
print(full_df.tail())

    id                                               text  HS  TR  AG
0  201  Hurray, saving us $$$ in so many ways @potus @...   1   0   0
1  202  Why would young fighting age men be the vast m...   1   0   0
2  203  @KamalaHarris Illegals Dump their Kids at the ...   1   0   0
3  204  NY Times: 'Nearly All White' States Pose 'an A...   0   0   0
4  205  Orban in Brussels: European leaders are ignori...   0   0   0
         id                                               text  HS  TR  AG
2995  31368  you can never take a L off a real bitch😩 im ho...   1   1   0
2996  30104  @Brian_202 likes to call me a cunt & a bitch b...   1   1   0
2997  31912  @kusha1a @Camio_the_wise @shoe0nhead 1. Never ...   1   1   0
2998  31000  If i see and know you a hoe why would i hit yo...   1   1   0
2999  30870   You be chasing them hoes fuck what a bitch think   1   1   0


In [4]:
import re

def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    return tweet

df['tweet'] = df["text"].apply(remove_usernames_links)
print(df.head())
print(df.tail())

                                                text                                              tweet
0  Hurray, saving us $$$ in so many ways @potus @...  Hurray, saving us $$$ in so many ways   #LockT...
1  Why would young fighting age men be the vast m...  Why would young fighting age men be the vast m...
2  @KamalaHarris Illegals Dump their Kids at the ...   Illegals Dump their Kids at the border like R...
3  NY Times: 'Nearly All White' States Pose 'an A...  NY Times: 'Nearly All White' States Pose 'an A...
4  Orban in Brussels: European leaders are ignori...  Orban in Brussels: European leaders are ignori...
                                                   text                                              tweet
2995  you can never take a L off a real bitch😩 im ho...  you can never take a L off a real bitch😩 im ho...
2996  @Brian_202 likes to call me a cunt & a bitch b...   likes to call me a cunt & a bitch but I tell ...
2997  @kusha1a @Camio_the_wise @shoe0nhead 1. Never ...

In [5]:
df["text_lower"] = df["tweet"].str.lower()
print(df.head())
print(df.tail())

                                                text  ...                                         text_lower
0  Hurray, saving us $$$ in so many ways @potus @...  ...  hurray, saving us $$$ in so many ways   #lockt...
1  Why would young fighting age men be the vast m...  ...  why would young fighting age men be the vast m...
2  @KamalaHarris Illegals Dump their Kids at the ...  ...   illegals dump their kids at the border like r...
3  NY Times: 'Nearly All White' States Pose 'an A...  ...  ny times: 'nearly all white' states pose 'an a...
4  Orban in Brussels: European leaders are ignori...  ...  orban in brussels: european leaders are ignori...

[5 rows x 3 columns]
                                                   text  ...                                         text_lower
2995  you can never take a L off a real bitch😩 im ho...  ...  you can never take a l off a real bitch😩 im ho...
2996  @Brian_202 likes to call me a cunt & a bitch b...  ...   likes to call me a cunt & a bitch but

In [6]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df['text_lower'].apply(lambda text: remove_punctuation(text))
df.head()

,text,tweet,text_lower,text_wo_punct
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...
3,NY Times: 'Nearly All White' States Pose 'an A...,NY Times: 'Nearly All White' States Pose 'an A...,ny times: 'nearly all white' states pose 'an a...,ny times nearly all white states pose an array...
4,Orban in Brussels: European leaders are ignori...,Orban in Brussels: European leaders are ignori...,orban in brussels: european leaders are ignori...,orban in brussels european leaders are ignorin...


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,text,tweet,text_lower,text_wo_punct,text_wo_stop
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...
3,NY Times: 'Nearly All White' States Pose 'an A...,NY Times: 'Nearly All White' States Pose 'an A...,ny times: 'nearly all white' states pose 'an a...,ny times nearly all white states pose an array...,ny times nearly white states pose array proble...
4,Orban in Brussels: European leaders are ignori...,Orban in Brussels: European leaders are ignori...,orban in brussels: european leaders are ignori...,orban in brussels european leaders are ignorin...,orban brussels european leaders ignoring peopl...


In [9]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
df["text_wo_urlno"] = df["text_wo_stop"].apply(lambda text: remove_urls(text))
df.head()

,text,tweet,text_lower,text_wo_punct,text_wo_stop,text_wo_urlno
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...,would young fighting age men vast majority one...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...
3,NY Times: 'Nearly All White' States Pose 'an A...,NY Times: 'Nearly All White' States Pose 'an A...,ny times: 'nearly all white' states pose 'an a...,ny times nearly all white states pose an array...,ny times nearly white states pose array proble...,ny times nearly white states pose array proble...
4,Orban in Brussels: European leaders are ignori...,Orban in Brussels: European leaders are ignori...,orban in brussels: european leaders are ignori...,orban in brussels european leaders are ignorin...,orban brussels european leaders ignoring peopl...,orban brussels european leaders ignoring peopl...


In [10]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

df["text_wo_htmlno"] = df["text_wo_urlno"].apply(lambda text: remove_html(text))
df.head()

,text,tweet,text_lower,text_wo_punct,text_wo_stop,text_wo_urlno,text_wo_htmlno
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...,would young fighting age men vast majority one...,would young fighting age men vast majority one...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...
3,NY Times: 'Nearly All White' States Pose 'an A...,NY Times: 'Nearly All White' States Pose 'an A...,ny times: 'nearly all white' states pose 'an a...,ny times nearly all white states pose an array...,ny times nearly white states pose array proble...,ny times nearly white states pose array proble...,ny times nearly white states pose array proble...
4,Orban in Brussels: European leaders are ignori...,Orban in Brussels: European leaders are ignori...,orban in brussels: european leaders are ignori...,orban in brussels european leaders are ignorin...,orban brussels european leaders ignoring peopl...,orban brussels european leaders ignoring peopl...,orban brussels european leaders ignoring peopl...


In [11]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text_wo_htmlno"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,text,tweet,text_lower,text_wo_punct,text_wo_stop,text_wo_urlno,text_wo_htmlno,text_lemmatized
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...,hurray save u many way lockthemup buildthewall...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...,would young fighting age men vast majority one...,would young fighting age men vast majority one...,would young fight age men vast majority one es...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...,illegals dump kid border like road kill refuse...
3,NY Times: 'Nearly All White' States Pose 'an A...,NY Times: 'Nearly All White' States Pose 'an A...,ny times: 'nearly all white' states pose 'an a...,ny times nearly all white states pose an array...,ny times nearly white states pose array proble...,ny times nearly white states pose array proble...,ny times nearly white states pose array proble...,ny time nearly white state pose array problem ...
4,Orban in Brussels: European leaders are ignori...,Orban in Brussels: European leaders are ignori...,orban in brussels: european leaders are ignori...,orban in brussels european leaders are ignorin...,orban brussels european leaders ignoring peopl...,orban brussels european leaders ignoring peopl...,orban brussels european leaders ignoring peopl...,orban brussels european leader ignore people w...


In [12]:
input_text = df["text_lemmatized"]
print(input_text)
print(type(input_text))

0       hurray save u many way lockthemup buildthewall...
1       would young fight age men vast majority one es...
2       illegals dump kid border like road kill refuse...
3       ny time nearly white state pose array problem ...
4       orban brussels european leader ignore people w...
                              ...                        
2995      never take l real bitch😩 im hotter hoe chill w😂
2996          like call cunt bitch tell truth cant handle
2997    1 never say taught 2 call bitch first 3 dont f...
2998    see know hoe would hit back lol bitch get new ...
2999                           chase hoe fuck bitch think
Name: text_lemmatized, Length: 12000, dtype: object
<class 'pandas.core.series.Series'>


In [13]:
#input_text converted to list --- THIS IS NEEDED FOR FITTING
input_text = input_text.tolist()
print(type(input_text))

<class 'list'>


In [15]:
Y = full_df.filter(['HS','TR','AG'])
print(Y)
print(type(Y))

      HS  TR  AG
0      1   0   0
1      1   0   0
2      1   0   0
3      0   0   0
4      0   0   0
...   ..  ..  ..
2995   1   1   0
2996   1   1   0
2997   1   1   0
2998   1   1   0
2999   1   1   0

[12000 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>


In [17]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(input_text,Y,test_size=0.2)
print(len(Train_X))
print(len(Test_X))
print(type(Test_X))
print(type(Train_X))
print(Train_Y)
print(Test_Y)
print(type(Train_Y))
print(type(Test_Y))

9600
2400
<class 'list'>
<class 'list'>
      HS  TR  AG
187    0   0   0
2082   0   0   0
4434   0   0   0
2151   0   0   0
1422   0   0   0
...   ..  ..  ..
534    1   0   0
3304   0   0   0
6476   0   0   0
1792   0   0   0
6599   0   0   0

[9600 rows x 3 columns]
      HS  TR  AG
8804   1   1   1
7005   0   0   0
5693   1   1   0
1205   0   0   0
7767   0   0   0
...   ..  ..  ..
546    1   0   1
6106   1   0   0
7915   0   0   0
7201   0   0   0
203    1   0   1

[2400 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
Countvectorizer = CountVectorizer(stop_words= 'english')
Countvectorizer.fit(input_text)

CountVectorizer(stop_words='english')

In [19]:
cbow_wm_trainX = Countvectorizer.transform(Train_X)
cbow_wm_testX = Countvectorizer.transform(Test_X)

In [23]:
print(cbow_wm_trainX)
print(type(cbow_wm_trainX))

  (0, 341)	1
  (0, 938)	1
  (0, 2431)	1
  (0, 3155)	1
  (0, 3272)	1
  (0, 4481)	1
  (0, 5492)	1
  (0, 10409)	1
  (0, 10414)	1
  (0, 12043)	1
  (0, 13024)	1
  (0, 13603)	1
  (0, 18306)	1
  (0, 18408)	1
  (1, 2238)	1
  (1, 4304)	1
  (1, 6014)	2
  (1, 16527)	2
  (1, 17776)	1
  (1, 18454)	1
  (2, 8353)	1
  (2, 9928)	1
  (2, 10078)	1
  (2, 10250)	1
  (2, 12832)	1
  :	:
  (9596, 17676)	1
  (9596, 17822)	1
  (9597, 2063)	1
  (9597, 4198)	1
  (9597, 5540)	1
  (9597, 5658)	1
  (9597, 5670)	1
  (9597, 6930)	1
  (9597, 11331)	1
  (9597, 12866)	1
  (9597, 13413)	1
  (9598, 2238)	1
  (9598, 4211)	1
  (9598, 6688)	1
  (9598, 7955)	1
  (9599, 289)	1
  (9599, 830)	1
  (9599, 875)	1
  (9599, 1300)	1
  (9599, 1503)	1
  (9599, 10542)	1
  (9599, 12974)	1
  (9599, 13388)	2
  (9599, 13841)	1
  (9599, 15421)	1
<class 'scipy.sparse.csr.csr_matrix'>


In [24]:
view_cbow_wm_trainX = cbow_wm_trainX.toarray()
view_cbow_wm_testX = cbow_wm_testX.toarray()
print(view_cbow_wm_trainX)
print(view_cbow_wm_trainX)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
feature_names = Countvectorizer.get_feature_names()
print(feature_names)
print(len(feature_names))

['000s', '0045', '0062', '007', '01', '040918', '047', '06', '07', '0827am', '0831', '10', '100', '1000', '10000', '100000', '1000000', '100000to', '10001', '1000month', '100100', '1004', '100th', '101', '1010', '1012', '1013', '102', '1024', '1030', '104', '106', '107', '107000', '109', '10am', '10am4pm', '10amangel', '10cm', '10k', '10mins', '10mn', '10month', '10th', '10tv', '10y', '10yearold', '10yrs', '11', '11000', '110000', '1112', '1115amjessica', '111yearold', '1124', '1130', '1130am', '114', '1159p', '1159pm', '1172', '118000', '11am', '11pm', '11yearold', '11yrold', '12', '120', '1200', '12000', '120k', '121', '122', '1221', '122ðÿ', '1230', '124', '12499636', '125', '12690crs', '127', '12746', '12deportthemallbuildthedamnwall', '12kyoto', '12p', '12yearold', '13', '130', '13000', '130000', '1325', '1325this', '133800', '135', '13785', '138', '139', '13th', '13yearold', '13yr', '14', '1400', '14000', '1425', '14300', '1488', '14th', '14yearold', '15', '150', '1500', '15000',

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
from sklearn import svm
from sklearn.metrics import accuracy_score
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
Train_Y_HS = Train_Y.filter(['HS'])
print(Train_Y_HS)
print(type(Train_Y_HS))

      HS
187    0
2082   0
4434   0
2151   0
1422   0
...   ..
534    1
3304   0
6476   0
1792   0
6599   0

[9600 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


In [31]:
SVM.fit(cbow_wm_trainX,Train_Y_HS)
predictions_SVM = SVM.predict(cbow_wm_testX)
print(predictions_SVM)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1 0 0 ... 0 1 0]


In [37]:
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y.filter(['HS']))*100)
expected_Y = Test_Y.filter(['HS'])
confusionMatrix_HS_SVM = confusion_matrix(expected_Y, predictions_SVM)
print(confusionMatrix_HS_SVM)
TP_HS_SVM = confusionMatrix_HS_SVM[0][0]
TN_HS_SVM = confusionMatrix_HS_SVM[1][1]
FP_HS_SVM = confusionMatrix_HS_SVM[0][1]
FN_HS_SVM = confusionMatrix_HS_SVM[1][0]
total_HS_SVM = TP_HS_SVM + TN_HS_SVM + FP_HS_SVM +FN_HS_SVM

print("true positive : ", TP_HS_SVM)
print("true negative : ", TN_HS_SVM)
print("false positive: ", FP_HS_SVM)
print("false negative: ", FN_HS_SVM)

print("Accuracy: ", ((TP_HS_SVM + TN_HS_SVM)/total_HS_SVM)*100,'%')
print("Recall or sensitivity : ", (TP_HS_SVM)/(TP_HS_SVM + FN_HS_SVM))
recall_HS_SVM = (TP_HS_SVM)/(TP_HS_SVM + FN_HS_SVM)
print("Precision : ", TP_HS_SVM/(TP_HS_SVM+FP_HS_SVM))
prec_HS_SVM = TP_HS_SVM/(TP_HS_SVM+FP_HS_SVM)
print("F-measure : ", 2*recall_HS_SVM*prec_HS_SVM/(recall_HS_SVM +prec_HS_SVM))
print("specificity : ", TN_HS_SVM/(TN_HS_SVM+FP_HS_SVM))

SVM Accuracy Score ->  70.70833333333333
[[1021  344]
 [ 359  676]]
true positive :  1021
true negative :  676
false positive:  344
false negative:  359
Accuracy:  70.70833333333333 %
Recall or sensitivity :  0.7398550724637681
Precision :  0.747985347985348
F-measure :  0.7438979963570127
specificity :  0.6627450980392157


In [40]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier()
rfclf.fit(cbow_wm_trainX,Train_Y_HS )
predictiction_RF = rfclf.predict(cbow_wm_testX)
print(predictiction_RF)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


[1 0 1 ... 1 0 1]


In [41]:
print("RF Accuracy Score -> ",accuracy_score(predictiction_RF, Test_Y.filter(['HS']))*100)
confusionMatrix_HS_RF = confusion_matrix(expected_Y, predictiction_RF)
print(confusionMatrix_HS_RF)
TP_HS_RF = confusionMatrix_HS_RF[0][0]
TN_HS_RF = confusionMatrix_HS_RF[1][1]
FP_HS_RF = confusionMatrix_HS_RF[0][1]
FN_HS_RF = confusionMatrix_HS_RF[1][0]
total_HS_RF = TP_HS_RF + TN_HS_RF + FP_HS_RF +FN_HS_RF

print("true positive : ", TP_HS_RF)
print("true negative : ", TN_HS_RF)
print("false positive: ", FP_HS_RF)
print("false negative: ", FN_HS_RF)

print("Accuracy: ", (TP_HS_RF + TN_HS_RF)/total_HS_RF)
print("Recall or sensitivity : ",(TP_HS_RF)/(TP_HS_RF + FN_HS_SVM))
recall_HS_RF = (TP_HS_RF)/(TP_HS_RF + FN_HS_RF)
print("Precision : ", TP_HS_RF/(TP_HS_RF+FP_HS_RF))
prec_HS_RF = TP_HS_RF/(TP_HS_RF + FP_HS_RF)
print("F-measure : ", 2*recall_HS_RF *prec_HS_RF/(recall_HS_RF + prec_HS_RF))
print("specificity : ", TN_HS_RF/(TN_HS_RF + FP_HS_RF))

RF Accuracy Score ->  72.95833333333334
[[1043  322]
 [ 327  708]]
true positive :  1043
true negative :  708
false positive:  322
false negative:  327
Accuracy:  0.7295833333333334
Recall or sensitivity :  0.7439372325249644
Precision :  0.764102564102564
F-measure :  0.7627056672760513
specificity :  0.6873786407766991


In [42]:
full_df_hs = full_df.loc[full_df['HS']==1]
print(full_df_hs)
df_hs = full_df_hs[["text"]]
df_hs["text"] = df_hs["text"].astype(str)
print(full_df_hs.head())

         id                                               text  HS  TR  AG
0       201  Hurray, saving us $$$ in so many ways @potus @...   1   0   0
1       202  Why would young fighting age men be the vast m...   1   0   0
2       203  @KamalaHarris Illegals Dump their Kids at the ...   1   0   0
5       206  @KurtSchlichter LEGAL is. Not illegal. #BuildT...   1   0   0
11      212  @RealDonaldTrump @HouseGOP Illegals are dumpin...   1   0   1
...     ...                                                ...  ..  ..  ..
2995  31368  you can never take a L off a real bitch😩 im ho...   1   1   0
2996  30104  @Brian_202 likes to call me a cunt & a bitch b...   1   1   0
2997  31912  @kusha1a @Camio_the_wise @shoe0nhead 1. Never ...   1   1   0
2998  31000  If i see and know you a hoe why would i hit yo...   1   1   0
2999  30870   You be chasing them hoes fuck what a bitch think   1   1   0

[5043 rows x 5 columns]
     id                                               text  HS  TR  AG
0   

In [43]:
df_hs['tweet'] = df_hs["text"].apply(remove_usernames_links)
print(df_hs.head())

                                                 text                                              tweet
0   Hurray, saving us $$$ in so many ways @potus @...  Hurray, saving us $$$ in so many ways   #LockT...
1   Why would young fighting age men be the vast m...  Why would young fighting age men be the vast m...
2   @KamalaHarris Illegals Dump their Kids at the ...   Illegals Dump their Kids at the border like R...
5   @KurtSchlichter LEGAL is. Not illegal. #BuildT...              LEGAL is. Not illegal. #BuildThatWall
11  @RealDonaldTrump @HouseGOP Illegals are dumpin...    Illegals are dumping their kids heres o they...


In [44]:
df_hs["text_lower"] = df_hs["tweet"].str.lower()
print(df_hs.head())

                                                 text  ...                                         text_lower
0   Hurray, saving us $$$ in so many ways @potus @...  ...  hurray, saving us $$$ in so many ways   #lockt...
1   Why would young fighting age men be the vast m...  ...  why would young fighting age men be the vast m...
2   @KamalaHarris Illegals Dump their Kids at the ...  ...   illegals dump their kids at the border like r...
5   @KurtSchlichter LEGAL is. Not illegal. #BuildT...  ...              legal is. not illegal. #buildthatwall
11  @RealDonaldTrump @HouseGOP Illegals are dumpin...  ...    illegals are dumping their kids heres o they...

[5 rows x 3 columns]


In [45]:
df_hs["text_wo_punct"] = df_hs['text_lower'].apply(lambda text: remove_punctuation(text))
df_hs.head()


,text,tweet,text_lower,text_wo_punct
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...
5,@KurtSchlichter LEGAL is. Not illegal. #BuildT...,LEGAL is. Not illegal. #BuildThatWall,legal is. not illegal. #buildthatwall,legal is not illegal buildthatwall
11,@RealDonaldTrump @HouseGOP Illegals are dumpin...,Illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...


In [46]:
df_hs["text_wo_stop"] = df_hs["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df_hs.head()

,text,tweet,text_lower,text_wo_punct,text_wo_stop
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...
5,@KurtSchlichter LEGAL is. Not illegal. #BuildT...,LEGAL is. Not illegal. #BuildThatWall,legal is. not illegal. #buildthatwall,legal is not illegal buildthatwall,legal illegal buildthatwall
11,@RealDonaldTrump @HouseGOP Illegals are dumpin...,Illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals dumping kids heres get welfare aid us...


In [47]:
df_hs["text_wo_urlno"] = df_hs["text_wo_stop"].apply(lambda text: remove_urls(text))
df_hs.head()

,text,tweet,text_lower,text_wo_punct,text_wo_stop,text_wo_urlno
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...,would young fighting age men vast majority one...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...
5,@KurtSchlichter LEGAL is. Not illegal. #BuildT...,LEGAL is. Not illegal. #BuildThatWall,legal is. not illegal. #buildthatwall,legal is not illegal buildthatwall,legal illegal buildthatwall,legal illegal buildthatwall
11,@RealDonaldTrump @HouseGOP Illegals are dumpin...,Illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals dumping kids heres get welfare aid us...,illegals dumping kids heres get welfare aid us...


In [48]:
df_hs["text_wo_htmlno"] = df_hs["text_wo_urlno"].apply(lambda text: remove_html(text))
df_hs.head()

,text,tweet,text_lower,text_wo_punct,text_wo_stop,text_wo_urlno,text_wo_htmlno
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...,would young fighting age men vast majority one...,would young fighting age men vast majority one...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...
5,@KurtSchlichter LEGAL is. Not illegal. #BuildT...,LEGAL is. Not illegal. #BuildThatWall,legal is. not illegal. #buildthatwall,legal is not illegal buildthatwall,legal illegal buildthatwall,legal illegal buildthatwall,legal illegal buildthatwall
11,@RealDonaldTrump @HouseGOP Illegals are dumpin...,Illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals dumping kids heres get welfare aid us...,illegals dumping kids heres get welfare aid us...,illegals dumping kids heres get welfare aid us...


In [49]:
df_hs["text_lemmatized"] = df_hs["text_wo_htmlno"].apply(lambda text: lemmatize_words(text))
df_hs.head()

,text,tweet,text_lower,text_wo_punct,text_wo_stop,text_wo_urlno,text_wo_htmlno,text_lemmatized
0,"Hurray, saving us $$$ in so many ways @potus @...","Hurray, saving us $$$ in so many ways #LockT...","hurray, saving us $$$ in so many ways #lockt...",hurray saving us in so many ways lockthemup...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...,hurray saving us many ways lockthemup buildthe...,hurray save u many way lockthemup buildthewall...
1,Why would young fighting age men be the vast m...,Why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,why would young fighting age men be the vast m...,would young fighting age men vast majority one...,would young fighting age men vast majority one...,would young fighting age men vast majority one...,would young fight age men vast majority one es...
2,@KamalaHarris Illegals Dump their Kids at the ...,Illegals Dump their Kids at the border like R...,illegals dump their kids at the border like r...,illegals dump their kids at the border like r...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...,illegals dump kids border like road kill refus...,illegals dump kid border like road kill refuse...
5,@KurtSchlichter LEGAL is. Not illegal. #BuildT...,LEGAL is. Not illegal. #BuildThatWall,legal is. not illegal. #buildthatwall,legal is not illegal buildthatwall,legal illegal buildthatwall,legal illegal buildthatwall,legal illegal buildthatwall,legal illegal buildthatwall
11,@RealDonaldTrump @HouseGOP Illegals are dumpin...,Illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals are dumping their kids heres o they...,illegals dumping kids heres get welfare aid us...,illegals dumping kids heres get welfare aid us...,illegals dumping kids heres get welfare aid us...,illegals dump kid here get welfare aid u schoo...


In [50]:
input_text_hs = df_hs["text_lemmatized"]
print(input_text_hs)
input_text_hs = input_text_hs.tolist()
print(input_text_hs)


0       hurray save u many way lockthemup buildthewall...
1       would young fight age men vast majority one es...
2       illegals dump kid border like road kill refuse...
5                             legal illegal buildthatwall
11      illegals dump kid here get welfare aid u schoo...
                              ...                        
2995      never take l real bitch😩 im hotter hoe chill w😂
2996          like call cunt bitch tell truth cant handle
2997    1 never say taught 2 call bitch first 3 dont f...
2998    see know hoe would hit back lol bitch get new ...
2999                           chase hoe fuck bitch think
Name: text_lemmatized, Length: 5043, dtype: object
['hurray save u many way lockthemup buildthewall enddaca boycottnfl boycottnike', 'would young fight age men vast majority one escape war amp cannot fight like woman child elderlyits majority refugees actually refugees economic migrant try get europe', 'illegals dump kid border like road kill refuse unite hope

In [51]:
Y_hs = full_df_hs.filter(['HS','TR','AG'])
print(Y_hs)

      HS  TR  AG
0      1   0   0
1      1   0   0
2      1   0   0
5      1   0   0
11     1   0   1
...   ..  ..  ..
2995   1   1   0
2996   1   1   0
2997   1   1   0
2998   1   1   0
2999   1   1   0

[5043 rows x 3 columns]


In [53]:
Train_X_hs, Test_X_hs, Train_Y_hs, Test_Y_hs = model_selection.train_test_split(input_text_hs,Y_hs,test_size=0.2)
Countvectorizer.fit(input_text_hs)
cbow_wm_trainX_hs = Countvectorizer.transform(Train_X_hs)
cbow_wm_testX_hs = Countvectorizer.transform(Test_X_hs)

In [54]:
Train_Y_TR_hs = Train_Y_hs.filter(['TR'])

      TR
3582   0
2449   0
5327   0
448    1
2610   1
...   ..
5212   1
4039   0
5535   0
895    0
5890   1

[4034 rows x 1 columns]


#SVM_TR

In [70]:
SVM.fit(cbow_wm_trainX_hs,Train_Y_TR_hs)
predictions_SVM_TR = SVM.predict(cbow_wm_testX_hs)
expected_Y_hstr = Test_Y_hs.filter(['TR'])
confusionMatrix_TR_SVM = confusion_matrix(expected_Y_hstr, predictions_SVM_TR)
print(confusionMatrix_TR_SVM)
TP_TR_SVM = confusionMatrix_TR_SVM[0][0]
TN_TR_SVM = confusionMatrix_TR_SVM[1][1]
FP_TR_SVM = confusionMatrix_TR_SVM[0][1]
FN_TR_SVM = confusionMatrix_TR_SVM[1][0]
total_TR_SVM = TP_TR_SVM + TN_TR_SVM + FP_TR_SVM +FN_TR_SVM

print("true positive : ", TP_TR_SVM)
print("true negative : ", TN_TR_SVM)
print("false positive: ", FP_TR_SVM)
print("false negative: ", FN_TR_SVM)

print("Accuracy: ", (TP_TR_SVM + TN_TR_SVM)/total_TR_SVM)
print("Recall or sensitivity : ",(TP_TR_SVM)/(TP_TR_SVM + FN_TR_SVM))
recall_TR_SVM = (TP_TR_SVM)/(TP_TR_SVM + FN_TR_SVM)
print("Precision : ", TP_TR_SVM/(TP_TR_SVM+FP_TR_SVM))
prec_TR_SVM = TP_TR_SVM/(TP_TR_SVM+FP_TR_SVM)
print("F-measure : ", 2*recall_TR_SVM*prec_TR_SVM/(recall_TR_SVM +prec_TR_SVM))
print("specificity : ", TN_TR_SVM/(TN_TR_SVM+FP_TR_SVM))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[514 118]
 [ 81 296]]
true positive :  514
true negative :  296
false positive:  118
false negative:  81
Accuracy:  0.8027750247770069
Recall or sensitivity :  0.8638655462184874
Precision :  0.8132911392405063
F-measure :  0.8378158109209455
specificity :  0.714975845410628


#SVM_AG

In [69]:
Train_Y_AG_hs = Train_Y_hs.filter(['AG'])
SVM.fit(cbow_wm_trainX_hs,Train_Y_AG_hs)
predictions_SVM_AG = SVM.predict(cbow_wm_testX_hs)
expected_Y_hsag = Test_Y_hs.filter(['AG'])
confusionMatrix_AG_SVM = confusion_matrix(expected_Y_hsag, predictions_SVM_AG)
print(confusionMatrix_AG_SVM)
TP_AG_SVM = confusionMatrix_AG_SVM[0][0]
TN_AG_SVM = confusionMatrix_AG_SVM[1][1]
FP_AG_SVM = confusionMatrix_AG_SVM[0][1]
FN_AG_SVM = confusionMatrix_AG_SVM[1][0]
total_AG_SVM = TP_AG_SVM + TN_AG_SVM + FP_AG_SVM +FN_AG_SVM

print("true positive : ", TP_AG_SVM)
print("true negative : ", TN_AG_SVM)
print("false positive: ", FP_AG_SVM)
print("false negative: ", FN_AG_SVM)

print("Accuracy: ", (TP_AG_SVM + TN_AG_SVM)/total_AG_SVM)
print("Recall or sensitivity : ",(TP_AG_SVM)/(TP_AG_SVM + FN_AG_SVM))
recall_AG_SVM = (TP_AG_SVM)/(TP_AG_SVM + FN_AG_SVM)
print("Precision : ", TP_AG_SVM/(TP_AG_SVM+FP_AG_SVM))
prec_AG_SVM = TP_AG_SVM/(TP_AG_SVM+FP_AG_SVM)
print("F-measure : ", 2*recall_AG_SVM*prec_AG_SVM/(recall_AG_SVM +prec_AG_SVM))
print("specificity : ", TN_AG_SVM/(TN_AG_SVM+FP_AG_SVM))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[443 148]
 [199 219]]
true positive :  443
true negative :  219
false positive:  148
false negative:  199
Accuracy:  0.6560951437066402
Recall or sensitivity :  0.6900311526479751
Precision :  0.7495769881556683
F-measure :  0.7185725871857258
specificity :  0.5967302452316077


# RF-TR

In [64]:
rfclf = RandomForestClassifier()
rfclf.fit(cbow_wm_trainX_hs,Train_Y_TR_hs )
predictiction_RF_TR = rfclf.predict(cbow_wm_testX_hs)
confusionMatrix_TR_RF = confusion_matrix(expected_Y_hstr, predictiction_RF_TR)
print(confusionMatrix_TR_RF)
TP_TR_RF = confusionMatrix_TR_RF[0][0]
TN_TR_RF = confusionMatrix_TR_RF[1][1]
FP_TR_RF = confusionMatrix_TR_RF[0][1]
FN_TR_RF = confusionMatrix_TR_RF[1][0]
total_TR_RF = TP_TR_RF + TN_TR_RF + FP_TR_RF +FN_TR_RF

print("true positive : ", TP_TR_RF)
print("true negative : ", TN_TR_RF)
print("false positive: ", FP_TR_RF)
print("false negative: ", FN_TR_RF)

print("Accuracy: ", (TP_TR_RF + TN_TR_RF)/total_TR_RF)
print("Recall or sensitivity : ",(TP_TR_RF)/(TP_TR_RF + FN_TR_SVM))
recall_TR_RF = (TP_TR_RF)/(TP_TR_RF + FN_TR_RF)
print("Precision : ", TP_TR_RF/(TP_TR_RF+FP_TR_RF))
prec_TR_RF = TP_TR_RF/(TP_TR_RF + FP_TR_RF)
print("F-measure : ", 2*recall_TR_RF *prec_TR_RF/(recall_TR_RF + prec_TR_RF))
print("specificity : ", TN_TR_RF/(TN_TR_RF + FP_TR_RF))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[[533  99]
 [ 61 316]]
true positive :  533
true negative :  316
false positive:  99
false negative:  61
Accuracy:  0.8414271555996036
Recall or sensitivity :  0.8680781758957655
Precision :  0.8433544303797469
F-measure :  0.869494290375204
specificity :  0.7614457831325301


#RF_AG

In [67]:
rfclf = RandomForestClassifier()
rfclf.fit(cbow_wm_trainX_hs,Train_Y_AG_hs )
predictiction_RF_AG = rfclf.predict(cbow_wm_testX_hs)
confusionMatrix_AG_RF = confusion_matrix(expected_Y_hsag, predictiction_RF_AG)
print(confusionMatrix_AG_RF)
TP_AG_RF = confusionMatrix_AG_RF[0][0]
TN_AG_RF = confusionMatrix_AG_RF[1][1]
FP_AG_RF = confusionMatrix_AG_RF[0][1]
FN_AG_RF = confusionMatrix_AG_RF[1][0]
total_AG_RF = TP_AG_RF + TN_AG_RF + FP_AG_RF +FN_AG_RF

print("true positive : ", TP_AG_RF)
print("true negative : ", TN_AG_RF)
print("false positive: ", FP_AG_RF)
print("false negative: ", FN_AG_RF)

print("Accuracy: ", (TP_AG_RF + TN_AG_RF)/total_AG_RF)
print("Recall or sensitivity : ",(TP_AG_RF)/(TP_AG_RF + FN_AG_SVM))
recall_AG_RF = (TP_AG_RF)/(TP_AG_RF + FN_AG_RF)
print("Precision : ", TP_AG_RF/(TP_TR_RF+FP_TR_RF))
prec_TR_RF = TP_TR_RF/(TP_TR_RF + FP_TR_RF)
print("F-measure : ", 2*recall_TR_RF *prec_TR_RF/(recall_TR_RF + prec_TR_RF))
print("specificity : ", TN_TR_RF/(TN_TR_RF + FP_TR_RF))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[[456 135]
 [184 234]]
true positive :  456
true negative :  234
false positive:  135
false negative:  184
Accuracy:  0.6838453914767096
Recall or sensitivity :  0.6961832061068702
Precision :  0.7215189873417721
F-measure :  0.869494290375204
specificity :  0.7614457831325301
